In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt as sqrt

In [2]:
#
diametre = np.array([15, 20, 17.5, 23])
#vom folosi 15 diametru standard pentru biesele la care analizam pierderea locala de sarcina
#vom folosi restul diametrelor pentru masuratorile pierderii liniare
#am creat o lista de labeluri pe care le vom folosi la grafice
labels = np.array([f'{i} mm' for i in diametre])

#datele experimentale pierdere liniara de sarcina

debite_pierdere_liniara ={'1' : np.array([0.880, 0.844, 0.742, 0.608, 0.471, 0.403, 0.323, 0.293, 0.107]),
                          '2' : np.array([1.196, 0.996, 0.882, 0.805, 0.701, 0.640, 0.547, 0.405, 0.330]),
                          '3' : np.array([1.075, 0.917, 0.718, 0.692, 0.613, 0.516, 0.390, 0.238, 0.233]), 
                          '4' : np.array([1.24, 1.05, 0.88, 0.80, 0.79, 0.69, 0.60, 0.5, 0.4])
                         }
presiuni_pierdere_liniara = {'1' : np.array([148.4, 135.13, 107.67, 75.38, 47.53, 35.64, 23.60, 20.07, 2.58]),
                             '2' : np.array([43.32, 29.58, 24.75, 20.46, 15.58, 13.32, 9.79, 5.42, 3.58]),
                             '3' : np.array([89.95, 66.33, 49.07, 39.17, 31.04, 22.68, 13.25, 4.81, 4.5]),
                             '4' : np.array([35.3, 25.3, 18.5, 14.8, 15.1, 11.5, 8.8, 5.81, 3.30])
                            }

#datele experimentale pierdere locala de saricna

debite_pierdere_locala = {'curba 90': np.array([0.509, 0.446, 0.398, 0.345, 0.302, 0.238, 0.199]),
                          'curba 45': np.array([0.60, 0.542, 0.509, 0.449, 0.407, 0.320, 0.232]),
                          'cot 135': np.array([0.511, 0.438, 0.390, 0.362, 0.310, 0.261]),
                          'cot 90':  np.array([0.603, 0.549, 0.500, 0.440, 0.391, 0.334, 0.300, 0.136])
                         }

presiuni_pierdere_locala = {'curba 90': np.array([1653, 1324.24, 1036.97, 763.29, 559.17, 311.50, 158.3]),
                            'curba 45': np.array([22.91, 18.77, 17.08, 13.71, 10.64, 7.34, 3.50]),
                            'cot 135': np.array([273.13, 200.49, 160.52, 139.12, 101, 71.47]),
                            'cot 90': np.array([204.86, 168.73, 140.89, 109.90, 85.89, 65.56, 49.37, 7.88])           
                           }

Zeta are formula: $\zeta=\frac{\Delta p D^4}{0.0826 \rho g L Q^2}$

Lambda are formula: $\lambda=\frac{\Delta p D^5}{0.0826 \rho g L Q^2}$

Folosim pentru Criteriul Reynolds formula: $Re = \frac{4Q}{\nu \pi D}$

Folosim pentru pierderea de sarcina formula $ h = \frac{\Delta p}{\rho g}$

In [18]:
class aferim:
    def __init__(self, presiune, debit, diametru, key = None):
        self.presiune = presiune * pow(10, 2) #conversie mbat --> bar
        self.debit    = debit    * pow(10,-3) #comversie  dm^3 --> m^3
        self.diametru = diametru * pow(10,-3) #conversie mm -> m
        self.key = key
    def __repr__(self):
        return f'{self.presiune}, {self.debit}, {self.diametru}'
    
    def lamda(self, ro = 1000, g = 9.81, L = 1):
        return self.presiune*pow(self.diametru, 5) / ( 0.0826*ro*g*L*pow(self.debit,2) )
    def zeta(self, ro = 1000, g = 9.81, L = 1):
        return self.presiune*pow(self.diametru, 4) / ( 0.0826*ro*g*L*pow(self.debit,2) )
    
    def reynolds(self, niu = pow(10, -6) ):
        return 4*self.debit / (np.pi*niu*self.diametru)    
    def pierdere_sarcina(self, ro = 1000, g = 10):
        return self.presiune / (ro*g)

    def df_liniar (self):
        data ={'Q (m^3/s)': self.debit,
               '\u0394p (Pa)': self.presiune,
               'D(m)': self.diametru,
               '\u03BB': self.lamda(), #lamda
               'Re': self.reynolds(), 
               'S': self.key
              }
        df = pd.DataFrame(data)
        return df

    def df_local (self):
        data ={'Q (m^3/s)': self.debit,
               '\u0394p (Pa)': self.presiune,
               'D(m)': self.diametru,
               '\u03B6': self.zeta(), #zeta
               'Re': self.reynolds(),
               'Obs': self.key
              }
        df = pd.DataFrame(data)
        return df

Pierderi Liniare

In [20]:
#s_1 = aferim(debit = debite_pierdere_liniara['1'], presiune = presiuni_pierdere_liniara['1'], diametru = diametre[0])
#print(s_1.df_lamda()) #testcase

masuratori_pierdere_liniara = []
for key in debite_pierdere_liniara.keys():
    i = 0
    obiect = aferim(debit = debite_pierdere_liniara[key], presiune = presiuni_pierdere_liniara[key], diametru = diametre[i], key = key)
    masuratori_pierdere_liniara.append(obiect)
    i += 1
    
for i in masuratori_pierdere_liniara:
    print(i.df_liniar())

   Q (m^3/s)  Δp (Pa)   D(m)         λ            Re  S
0   0.000880  14840.0  0.015  0.017959  74696.719958  1
1   0.000844  13513.0  0.015  0.017778  71640.945050  1
2   0.000742  10767.0  0.015  0.018327  62982.916146  1
3   0.000608   7538.0  0.015  0.019110  51608.642880  1
4   0.000471   4753.0  0.015  0.020079  39979.721705  1
5   0.000403   3564.0  0.015  0.020565  34207.702435  1
6   0.000323   2360.0  0.015  0.021199  27417.091530  1
7   0.000293   2007.0  0.015  0.021909  24870.612440  1
8   0.000107    258.0  0.015  0.021118   9082.442086  1
   Q (m^3/s)  Δp (Pa)   D(m)         λ             Re  S
0   0.001196   4332.0  0.015  0.002838  101519.633034  2
1   0.000996   2958.0  0.015  0.002794   84543.105770  2
2   0.000882   2475.0  0.015  0.002982   74866.485230  2
3   0.000805   2046.0  0.015  0.002959   68330.522234  2
4   0.000701   1558.0  0.015  0.002971   59502.728057  2
5   0.000640   1332.0  0.015  0.003048   54324.887242  2
6   0.000547    979.0  0.015  0.003066   

Pierderi Locale

In [21]:
masuratori_pierdere_locala = []
#diametrul ramane constant la pierderea locala deoarece este acelasi pentru toate piesele
for key in debite_pierdere_locala.keys():
    obiect = aferim(debit = debite_pierdere_locala[key], presiune = presiuni_pierdere_locala[key], diametru = diametre[0], key = key)
    masuratori_pierdere_locala.append(obiect)

for i in masuratori_pierdere_locala:
    print(i.df_local())

   Q (m^3/s)   Δp (Pa)   D(m)          ζ            Re       Obs
0   0.000509  165300.0  0.015  39.861466  43205.261885  curba 90
1   0.000446  132424.0  0.015  41.592303  37857.655797  curba 90
2   0.000398  103697.0  0.015  40.899315  33783.289254  curba 90
3   0.000345   76329.0  0.015  40.065205  29284.509529  curba 90
4   0.000302   55917.0  0.015  38.304161  25634.556167  curba 90
5   0.000238   31150.0  0.015  34.357386  20202.067443  curba 90
6   0.000199   15830.0  0.015  24.974152  16891.644627  curba 90
   Q (m^3/s)  Δp (Pa)   D(m)         ζ            Re       Obs
0   0.000600   2291.0  0.015  0.397593  50929.581789  curba 45
1   0.000542   1877.0  0.015  0.399192  46006.388883  curba 45
2   0.000509   1708.0  0.015  0.411878  43205.261885  curba 45
3   0.000449   1371.0  0.015  0.424875  38112.303706  curba 45
4   0.000407   1064.0  0.015  0.401300  34547.232980  curba 45
5   0.000320    734.0  0.015  0.447829  27162.443621  curba 45
6   0.000232    350.0  0.015  0.406264 